## Módulo 1

In [1]:
#Importando as bibliotecas

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

iris = load_iris()
x = iris.data #[n_amostras, n_features]
y = iris.target # Vetor de alvo [n_amostras]

print("Forma das features (x): ", x.shape)
print("Forma das caracteristicas (x): ", x.shape[1])
print("Forma do valor alvo (y): ", y.shape)

Forma das features (x):  (150, 4)
Forma das caracteristicas (x):  4
Forma do valor alvo (y):  (150,)


## Treino e Teste

### Particionar os dados para treinamento

In [2]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=30)

print("Tamanho do conjunto do treinamento: ", x_train.shape)
print("Tamanho do conjunto do teste: ", x_test.shape)

Tamanho do conjunto do treinamento:  (105, 4)
Tamanho do conjunto do teste:  (45, 4)


### Árvore de decisão

In [3]:
tree_model = DecisionTreeClassifier(random_state=30)
tree_model.fit(x_train, y_train)
print(tree_model.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 30, 'splitter': 'best'}


### Predição

In [4]:
y_tree_predict = tree_model.predict(x_test)
accuracy_tree = accuracy_score(y_test, y_tree_predict)

print(accuracy_tree)

0.9555555555555556


## KNN K-Nearest Neighbors

### Escalonamento de Dados

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

knn_model = KNeighborsClassifier()
knn_model.fit(x_train_scaled, y_train)

y_pred_knn = knn_model.predict(x_test_scaled)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print(f'KNN: {accuracy_knn:.2f}')